* Importing the required modeuls and reading the raw data

In [1]:
import json
import requests
from urllib.request import urlopen
import time
import pandas as pd
import numpy as np
import glob
url = "https://data.cityofnewyork.us/api/views/uvpi-gqnh/rows.csv?accessType=DOWNLOAD"
original = pd.read_csv(url)
original.sample(5)

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
496956,707763,205047,09/11/2016,4,0,OnCurb,Alive,Good,Prunus,cherry,...,3040000,New York,40.597287,-73.974087,9.914462e+05,156885.2263,47.0,400.0,3191376.0,3.071020e+09
616245,122377,337506,08/06/2015,8,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,4130100,New York,40.731956,-73.740624,1.056135e+06,206054.4895,23.0,1301.0,4563146.0,4.079430e+09
11489,207484,319786,09/06/2015,3,0,OnCurb,Alive,Good,Quercus palustris,pin oak,...,4129104,New York,40.741180,-73.755033,1.052133e+06,209403.3289,23.0,129104.0,4444484.0,4.077420e+09
143733,337298,510003,10/16/2015,8,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,2009600,New York,40.837095,-73.846122,1.026830e+06,244290.9268,18.0,96.0,2103339.0,2.038460e+09
216365,406859,206142,10/30/2015,15,0,OnCurb,Alive,Good,Quercus palustris,pin oak,...,3026600,New York,40.612839,-73.989786,9.870858e+05,162550.4443,47.0,266.0,3158005.0,3.061840e+09


* Filtering out "Dead" trees according to the status column in the data

In [2]:
notDead = original[original.status != 'Dead']

* Selecting the appropriate columns and saving the results in a new file

In [3]:
SelectedColumns = ['tree_id', 'latitude', 'longitude']
final = notDead.loc[:, SelectedColumns]
final.to_csv('Tree_LatLong.csv')
final.sample(5)

,tree_id,latitude,longitude
435739,623942,40.675932,-73.812018
430780,655717,40.759579,-73.900920
200384,359925,40.537187,-74.204952
168389,369704,40.540392,-74.199399
521810,138,40.816356,-73.890581


* So far we have managed to have the exact location of each tree, but to generate the final dataset we must convert latitude and longitude to census tract. Function below is defined to take two arguments, latitude and longitude, as the input and return the FIPS code.

In [4]:
def geo_to_fips(latitude, longitude):
    url = "https://geo.fcc.gov/api/census/block/find?latitude=" + str(latitude) + "&longitude="+ str(longitude) + "&format=json"
    obj = json.load(urlopen(url))
    return  obj['Block']['FIPS']

* Note that the raw data is too big to use the API for all the data at once. It is suggested to break the data into smaller subsets and convert latitude and longitude to FIPS separately. Eventually the converted subsets need to be merged to create the final dataset.